In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_data=pd.read_csv('train.csv').fillna(-999)
test_data=pd.read_csv('test_noLabel.csv').fillna(-999)

In [2]:
def objectal(df):
    for col in df.columns:
        if col!='Label':
            df[col]   = df[col].astype(str)
    return df
train_data=objectal(train_data)
test_data=objectal(test_data)

mon1_train_df=train_data[train_data.loc[:,'时间']=='1'].drop(['时间','ID','小区房屋出租数量'],axis=1).drop_duplicates()
mon2_train_df=train_data[train_data.loc[:,'时间']=='2'].drop(['时间','ID','小区房屋出租数量'],axis=1).drop_duplicates()
mon3_train_df=train_data[train_data.loc[:,'时间']=='3'].drop(['时间','ID','小区房屋出租数量'],axis=1).drop_duplicates()

common_cols=list(mon1_train_df.columns)
common_cols.remove('Label')

# 按月计算出房屋的均租金
mon1_train_df=mon1_train_df.groupby(common_cols,as_index=False).mean()
mon2_train_df=mon2_train_df.groupby(common_cols,as_index=False).mean()
mon3_train_df=mon3_train_df.groupby(common_cols,as_index=False).mean()

# 二月并一月，缺失值由一月数据来填充
recent_mean_rental=mon2_train_df.merge(mon1_train_df,how='outer',on=common_cols).fillna(method='bfill',axis=1)
recent_mean_rental=recent_mean_rental.drop(['Label_y'],axis=1).rename(columns={'Label_x':'Label'})
# 三月并二月，缺失值由二月(一月)来填充
recent_mean_rental=mon3_train_df.merge(recent_mean_rental,how='outer',on=common_cols).fillna(method='bfill',axis=1)
recent_mean_rental=recent_mean_rental.drop(['Label_y'],axis=1).rename(columns={'Label_x':'Label'})

statistic_pred=test_data.merge(recent_mean_rental,how='left',on=common_cols)
statistic_pred.loc[:,['ID','Label']].to_csv('statistic_pred.csv',index=False)

In [3]:
statistic_pred['Label'].value_counts()

5.602716     2364
4.923599     1719
7.300509     1702
6.281834     1498
8.998302     1376
             ... 
15.223543       1
4.837012        1
9.677419        1
7.526882        1
22.920204       1
Name: Label, Length: 628, dtype: int64